In [53]:
import numpy as np
import pandas as pd
import os

os.environ["KERAS_BACKEND"] = "jax"

# Note that Keras should only be imported after the backend
# has been configured. The backend cannot be changed once the
# package is imported.
import keras
import tensorflow as tf

In [9]:
from google.colab import drive
drive.mount('/content/drive')

from os import getcwd
from os.path import join
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content'

In [29]:
# Load the data and split it between train and test sets
image_dir = join("drive", "MyDrive", "Colab Notebooks", "mel-spectograms")
ds = keras.preprocessing.image_dataset_from_directory(
    image_dir,
    image_size=(128,128),  # resize if needed
    batch_size=32,
    shuffle=True
)

Found 1166 files belonging to 4 classes.


In [54]:
# Split into train/test
train_size = int(0.8 * len(ds))
ds_train = ds.take(train_size)
ds_test = ds.skip(train_size)

ds_train = ds_train.map(lambda x, y: (tf.image.rgb_to_grayscale(tf.cast(x, tf.float32)/255.), y))
ds_test = ds_test.map(lambda x, y: (tf.image.rgb_to_grayscale(tf.cast(x, tf.float32)/255.), y))

# Optionally, create a validation dataset from training data
val_size = 0.15
total_batches = sum(1 for _ in ds_train)
val_batches = int(total_batches * val_size)

ds_val = ds_train.take(val_batches)
ds_train_final = ds_train.skip(val_batches)

In [55]:
# Model parameters
num_classes = 4
input_shape = (128, 128, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

In [56]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 124, 124, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 58, 58, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 259,524 (1013.77 KB)

 Trainable params: 259,524 (1013.77 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [58]:
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
]

model.fit(
    ds_train_final,
    validation_data=ds_val,
    epochs=epochs,
    callbacks=callbacks
)

score = model.evaluate(ds_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 199ms/step - acc: 0.2358 - loss: 1.3863 - val_acc: 0.2578 - val_loss: 1.3826
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - acc: 0.3227 - loss: 1.3785 - val_acc: 0.4062 - val_loss: 1.3711
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - acc: 0.3681 - loss: 1.3708 - val_acc: 0.4453 - val_loss: 1.3389
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - acc: 0.4424 - loss: 1.3247 - val_acc: 0.4453 - val_loss: 1.2393
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - acc: 0.3837 - loss: 1.2763 - val_acc: 0.3984 - val_loss: 1.2322
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - acc: 0.4256 - loss: 1.2117 - val_acc: 0.3516 - val_loss: 1.2125
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step - acc: 0.4205 - loss: 1.2129 - val_acc: 0.3906 - val_loss: 1.2444
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - acc: 0.3908 - loss: 1.2201 - val_acc: 0.4453 - val_loss: 1.1689
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - acc

In [59]:
model.save("model_customcnn.keras")